In [ ]:
class WTNode:
    def __init__(self,d,l,m,r):
        self.data = d
        self.left = l
        self.right = r
        self.midl = m
        self.mult = 0
          
    # prints the node and all its children in a string
    def __str__(self):  
        st = "("+str(self.data)+", "+str(self.mult)+") -> ["
        if self.left != None:
            st += str(self.left)
        else: st += "None"
        if self.midl != None:
            st += ", "+str(self.midl)
        else: st += ", None"
        if self.right != None:
            st += ", "+str(self.right)
        else: st += ", None"
        return st + "]"
    
class WordTree:
    def __init__(self):
        self.root = None
        self.size = 0
        
    # Holds data for a node, its parent and a flag representing
    # the link between them
    class NodeWrapper:
        def __init__(self, p, n, f):
            self.parent = p
            self.actual = n
            self.flag = f
        
    def __str__(self):
        return str(self.root)

    # returns the number of times that string st is stored in the tree
    def count(self, st):    
        if len(st) == 0: return
        ptr = self.root  
        i = 0
        while ptr != None:
            if st[i] == ptr.data:
                if i == len(st)-1: return ptr.mult
                ptr = ptr.midl
                i += 1
            elif st[i] < ptr.data:
                ptr = ptr.left
            elif st[i] > ptr.data:
                ptr = ptr.right
        return 0
        
    # adds the string st in the tree, increases the size by 1
    # and returns None
    def add(self, st):
        if len(st) == 0: return
        if self.size == 0: return self._initTree(st)
        flag = None; parent = None; currNode = self.root
        i = 0
        while i < len(st):
            if currNode == None:
                if flag == 'l':
                    parent.left = WTNode(st[i], None, None, None)
                    currNode = parent.left
                elif flag == 'm':
                    parent.midl = WTNode(st[i], None, None, None)
                    currNode = parent.midl
                elif flag == 'r':
                    parent.right = WTNode(st[i], None, None, None)
                    currNode = parent.right
                parent = currNode; currNode = currNode.midl
                flag = 'm'; i += 1
            elif st[i] == currNode.data:
                parent = currNode; currNode = currNode.midl
                flag = 'm'; i += 1
            elif st[i] > currNode.data:
                parent = currNode; currNode = currNode.right
                flag = 'r'
            elif st[i] < currNode.data:
                parent = currNode; currNode = currNode.left
                flag = 'l'
        parent.mult += 1
        self.size += 1
                
    # builds the first word in the tree
    def _initTree(self, st):
        self.root = WTNode(st[0], None, None, None)
        ptr = self.root
        st = st[1:]
        while st:
            ptr.midl = WTNode(st[0], None, None, None)
            st = st[1:]
            ptr = ptr.midl
        ptr.mult += 1
        self.size += 1

    # returns the lexicographically smallest string in the tree
    # if the tree is empty, return None
    def minst(self):    
        if self.size == 0: return
        res = ""; ptr = self.root
        while ptr.left != None or ptr.midl != None:
            if ptr.left != None:
                ptr = ptr.left
            elif ptr.midl != None:
                res += ptr.data
                if ptr.mult > 0: return res
                ptr = ptr.midl
        res += ptr.data              
        return res

    # removes one occurrence of string st from the tree and returns None
    # if st does not occur in the tree then it returns without changing the tree
    # it updates the size of the tree accordingly
    def remove(self,st):    
        if self.count(st) == 0 or len(st) == 0:
            return
        else:
            self._removeWord(st)
            self.size -= 1
            return
        
    def _removeWord(self, st):
        nodesToRemove = self._buildArrayOfWrappers(st)
        for i in range(len(nodesToRemove)-1, -1, -1):
            currNode = nodesToRemove[i].actual
            if i == len(nodesToRemove)-1:
                currNode.mult -= 1
                if currNode.mult > 0: return
            if currNode.mult == 0 and currNode.midl == None:
                self._removeNode(nodesToRemove[i])
                
    # Builds an array of type NodeWrapper
    def _buildArrayOfWrappers(self, st):
        A = [None for i in range(len(st))]
        i = 0
        parent = None
        ptr = self.root
        flag = None
        while len(st) > 0:
            if st[0] == ptr.data:
                A[i] = self.NodeWrapper(parent, ptr, flag)
                parent = ptr
                ptr = ptr.midl
                flag = 'm'
                st = st[1:]
                i += 1
            elif st[0] < ptr.data:
                parent = ptr
                ptr = ptr.left
                flag = 'l'
            elif st[0] > ptr.data:
                parent = ptr
                ptr = ptr.right
                flag = 'r'      
        return A
                
    # Removes a node
    def _removeNode(self, wrapper):
        parent = wrapper.parent
        currNode = wrapper.actual
        flag = wrapper.flag
        
        # If trying to remove root node
        if parent == None: return self._removeRoot()
        
        # If removing a node with no children - simply remove
        if currNode.left == None and currNode.right == None:
            if flag == 'l': parent.left = None
            elif flag == 'r': parent.right = None
            elif flag == 'm': parent.midl = None
        # If node has a child on the right only - simply bypass
        elif currNode.left == None:
            if flag == 'l': parent.left = currNode.right
            elif flag == 'r': parent.right = currNode.right
            elif flag == 'm': parent.midl = currNode.right
        # If node has a child on the left only - simply bypass
        elif currNode.right == None:
            if flag == 'l': parent.left = currNode.left
            elif flag == 'r': parent.right = currNode.left
            elif flag == 'm': parent.midl = currNode.left
        # If node has children on the left and the right
        # Find min node and remove it from current position, then put it in
        # the position of the node that is being removed
        else:
            if currNode.right.left == None:
                minNode = currNode.right
                minNode.left = currNode.left
            else:
                minNode = self._removeAndReturnMin(currNode)
                minNode.left = currNode.left
                minNode.right = currNode.right
            if flag == 'l': parent.left = minNode
            elif flag == 'r': parent.right = minNode
            elif flag == 'm': parent.midl = minNode
                
    # Find the min node by going right once, then left as far as possible
    def _removeAndReturnMin(self, currNode):
        ptr = currNode.right
        while ptr.left != None:
            parent = ptr
            ptr = ptr.left
        parent.left = None
        return ptr

    # Remove the root of the tree
    def _removeRoot(self):
        if self.root.left == None and self.root.right == None:
            self.root = None
        elif self.root.left == None:
            self.root = self.root.right
        elif self.root.right == None:
            self.root = self.root.left
        else:
            if self.root.right.left == None:
                minNode = self.root.right
                minNode.left = self.root.left
            else:
                minNode = self._removeAndReturnMin(self.root)
                minNode.left = self.root.left
                minNode.right = self.root.right
            self.root = minNode

In [4]:
import time
def test_message2(test,number,results,messages):
    msg = str("TESTING WordTree ["+test+"] FOR "+str(number)+" WORDS")
    print(''.join(map(str, ['*' for i in range(len(msg))])))
    print(msg)
    print("[printing results].........", end="", flush=True)
    success = True
    for i in range(len(results)):
        if not results[i]: 
            print("[error: "+messages[i]+"]")
            success = False
    if success: print("[success: passed all tests]")
    print()
    return success

words15  = ["carbos", "room", "pump", "run", "meter", "video", "car", "cart", "cart", "cage", "phone", "nil", "stat", "visas", "up"]
words100 = ["bick", "psychology", "none", "pam", "cajit", "pus", "rof", "zesh", "net", "leho", "bal", "spipres", "ror", "spix", "abrar", "ep", "cor", "stiocra", "enta", "siss", "blassa", "eratch", "kerjabra", "toprot", "huctaica", "beps", "horaf", "najing", "foofar", "nilu", "uck", "faegnols", "stopa", "pull", "cacrorruss", "escluwar", "smap", "ehut", "wisos", "strat", "fummehy", "puspro", "lar", "ron", "plenye", "hodjan", "raxa", "ditele", "bot", "fat", "nal", "frarut", "simb", "bar", "fiso", "genth", "cral", "vidzo", "uta", "keth", "pass", "ang", "stuja", "coll", "run", "vuir", "fess", "scunga", "zennaty", "od", "cor", "ritto", "er", "gomd", "run", "ustragga", "ming", "braeblo", "woildru", "hol", "westant", "clawa", "pe", "udlu", "an", "ral", "pus", "iumuosky", "scaroor", "latsa", "bandj", "nonnuck", "bather", "armitse", "stud", "sunts", "utlu", "cana", "tiur", "ain"]
wt15 = WordTree()
wt100 = WordTree()

#####################################################################################################
# TEST add.1)Check adding 15 words.2)Check adding "".3)Check adding 100 words.Check size and print. #
#####################################################################################################
def words15_add():
    print("[adding 15 words]...........", end="", flush=True)
    for w in words15:
        wt15.add(w)
    print("[success]")
    print("Min word in wt15 is: " + wt15.minst())
    return (
        str(wt15) == "(c, 0) -> [None, (a, 0) -> [None, (r, 1) -> [(g, 0) -> [None, (e, 1) -> [None, None, None], None], (b, 0) -> [None, (o, 0) -> [None, (s, 1) -> [None, None, None], None], (t, 2) -> [None, None, None]], None], None], (r, 0) -> [(p, 0) -> [(m, 0) -> [None, (e, 0) -> [None, (t, 0) -> [None, (e, 0) -> [None, (r, 1) -> [None, None, None], None], None], None], (n, 0) -> [None, (i, 0) -> [None, (l, 1) -> [None, None, None], None], None]], (u, 0) -> [(h, 0) -> [None, (o, 0) -> [None, (n, 0) -> [None, (e, 1) -> [None, None, None], None], None], None], (m, 0) -> [None, (p, 1) -> [None, None, None], None], None], None], (o, 0) -> [None, (o, 0) -> [None, (m, 1) -> [None, None, None], None], (u, 0) -> [None, (n, 1) -> [None, None, None], None]], (v, 0) -> [(s, 0) -> [None, (t, 0) -> [None, (a, 0) -> [None, (t, 1) -> [None, None, None], None], None], (u, 0) -> [None, (p, 1) -> [None, None, None], None]], (i, 0) -> [None, (d, 0) -> [None, (e, 0) -> [None, (o, 1) -> [None, None, None], None], (s, 0) -> [None, (a, 0) -> [None, (s, 1) -> [None, None, None], None], None]], None], None]]]",
        wt15.size == 15
    )
def words15_addEmpty():
    p = str(wt15)
    s = wt15.size
    wt15.add("")
    return (p == str(wt15) , s == wt15.size )
def words100_add():
    print("[adding 100 words]...........", end="", flush=True)
    for w in words100:
        wt100.add(w)
    print("[success]")
    print("Min word in wt100 is: " + wt100.minst())
    return (
        str(wt100) == "(b, 0) -> [(a, 0) -> [None, (b, 0) -> [None, (r, 0) -> [None, (a, 0) -> [None, (r, 1) -> [None, None, None], None], None], (n, 1) -> [(i, 0) -> [None, (n, 1) -> [None, None, None], None], (g, 1) -> [None, None, None], (r, 0) -> [None, (m, 0) -> [None, (i, 0) -> [None, (t, 0) -> [None, (s, 0) -> [None, (e, 1) -> [None, None, None], None], None], None], None], None]]], None], (i, 0) -> [(a, 0) -> [None, (l, 1) -> [None, None, (r, 1) -> [(n, 0) -> [None, (d, 0) -> [None, (j, 1) -> [None, None, None], None], None], None, (t, 0) -> [None, (h, 0) -> [None, (e, 0) -> [None, (r, 1) -> [None, None, None], None], None], None]]], (e, 0) -> [None, (p, 0) -> [None, (s, 1) -> [None, None, None], None], None]], (c, 0) -> [None, (k, 1) -> [None, None, None], None], (l, 0) -> [None, (a, 0) -> [None, (s, 0) -> [None, (s, 0) -> [None, (a, 1) -> [None, None, None], None], None], None], (o, 0) -> [None, (t, 1) -> [None, None, None], (r, 0) -> [None, (a, 0) -> [None, (e, 0) -> [None, (b, 0) -> [None, (l, 0) -> [None, (o, 1) -> [None, None, None], None], None], None], None], None]]]], (p, 0) -> [(n, 0) -> [(c, 0) -> [None, (a, 0) -> [None, (j, 0) -> [(c, 0) -> [None, (r, 0) -> [None, (o, 0) -> [None, (r, 0) -> [None, (r, 0) -> [None, (u, 0) -> [None, (s, 0) -> [None, (s, 1) -> [None, None, None], None], None], None], None], None], None], None], (i, 0) -> [None, (t, 1) -> [None, None, None], None], (n, 0) -> [None, (a, 1) -> [None, None, None], None]], (o, 0) -> [(l, 0) -> [None, (a, 0) -> [None, (w, 0) -> [None, (a, 1) -> [None, None, None], None], None], None], (r, 2) -> [(l, 0) -> [None, (l, 1) -> [None, None, None], None], None, None], (r, 0) -> [None, (a, 0) -> [None, (l, 1) -> [None, None, None], None], None]]], (l, 0) -> [(e, 0) -> [(d, 0) -> [None, (i, 0) -> [None, (t, 0) -> [None, (e, 0) -> [None, (l, 0) -> [None, (e, 1) -> [None, None, None], None], None], None], None], None], (p, 1) -> [(n, 0) -> [(h, 0) -> [None, (u, 0) -> [None, (t, 1) -> [None, None, None], None], None], (t, 0) -> [None, (a, 1) -> [None, None, None], None], None], None, (r, 1) -> [None, (a, 0) -> [None, (t, 0) -> [None, (c, 0) -> [None, (h, 1) -> [None, None, None], None], None], None], (s, 0) -> [None, (c, 0) -> [None, (l, 0) -> [None, (u, 0) -> [None, (w, 0) -> [None, (a, 0) -> [None, (r, 1) -> [None, None, None], None], None], None], None], None], None]]], (k, 0) -> [(h, 0) -> [(f, 0) -> [None, (o, 0) -> [(a, 0) -> [None, (e, 0) -> [None, (g, 0) -> [None, (n, 0) -> [None, (o, 0) -> [None, (l, 0) -> [None, (s, 1) -> [None, None, None], None], None], None], None], (t, 1) -> [None, None, None]], (i, 0) -> [(e, 0) -> [None, (s, 0) -> [None, (s, 1) -> [None, None, None], None], None], (s, 0) -> [None, (o, 1) -> [None, None, None], None], None]], (o, 0) -> [None, (f, 0) -> [None, (a, 0) -> [None, (r, 1) -> [None, None, None], None], None], None], (u, 0) -> [(r, 0) -> [None, (a, 0) -> [None, (r, 0) -> [None, (u, 0) -> [None, (t, 1) -> [None, None, None], None], None], None], None], (m, 0) -> [None, (m, 0) -> [None, (e, 0) -> [None, (h, 0) -> [None, (y, 1) -> [None, None, None], None], None], None], None], None]], (g, 0) -> [None, (e, 0) -> [None, (n, 0) -> [None, (t, 0) -> [None, (h, 1) -> [None, None, None], None], None], (o, 0) -> [None, (m, 0) -> [None, (d, 1) -> [None, None, None], None], None]], None]], (u, 0) -> [(o, 0) -> [None, (r, 0) -> [(d, 0) -> [None, (j, 0) -> [None, (a, 0) -> [None, (n, 1) -> [None, None, None], None], None], (l, 1) -> [None, None, None]], (a, 0) -> [None, (f, 1) -> [None, None, None], None], None], None], (c, 0) -> [None, (t, 0) -> [None, (a, 0) -> [None, (i, 0) -> [None, (c, 0) -> [None, (a, 1) -> [None, None, None], None], None], None], None], None], None], (i, 0) -> [None, (u, 0) -> [None, (m, 0) -> [None, (u, 0) -> [None, (o, 0) -> [None, (s, 0) -> [None, (k, 0) -> [None, (y, 1) -> [None, None, None], None], None], None], None], None], None], None]], (e, 0) -> [None, (r, 0) -> [None, (j, 0) -> [None, (a, 0) -> [None, (b, 0) -> [None, (r, 0) -> [None, (a, 1) -> [None, None, None], None], None], None], None], (t, 0) -> [None, (h, 1) -> [None, None, None], None]], None], None]], (e, 0) -> [(a, 0) -> [None, (r, 1) -> [None, None, (t, 0) -> [None, (s, 0) -> [None, (a, 1) -> [None, None, None], None], None]], None], (h, 0) -> [None, (o, 1) -> [None, None, None], None], None], (m, 0) -> [None, (i, 0) -> [None, (n, 0) -> [None, (g, 1) -> [None, None, None], None], None], None]]], (o, 0) -> [(e, 0) -> [(a, 0) -> [None, (j, 0) -> [None, (i, 0) -> [None, (n, 0) -> [None, (g, 1) -> [None, None, None], None], None], (l, 1) -> [None, None, None]], None], (t, 1) -> [None, None, None], (i, 0) -> [None, (l, 0) -> [None, (u, 1) -> [None, None, None], None], None]], (n, 0) -> [None, (e, 1) -> [None, None, (n, 0) -> [None, (u, 0) -> [None, (c, 0) -> [None, (k, 1) -> [None, None, None], None], None], None]], None], None], (o, 0) -> [None, (d, 1) -> [None, None, None], None]], (s, 0) -> [(a, 0) -> [None, (m, 1) -> [None, None, (s, 0) -> [None, (s, 1) -> [None, None, None], None]], (l, 0) -> [(e, 1) -> [None, None, None], (e, 0) -> [None, (n, 0) -> [None, (y, 0) -> [None, (e, 1) -> [None, None, None], None], None], None], None]], (y, 0) -> [None, (c, 0) -> [None, (h, 0) -> [None, (o, 0) -> [None, (l, 0) -> [None, (o, 0) -> [None, (g, 0) -> [None, (y, 1) -> [None, None, None], None], None], None], None], None], None], None], (u, 0) -> [None, (s, 2) -> [(l, 0) -> [None, (l, 1) -> [None, None, None], None], (p, 0) -> [None, (r, 0) -> [None, (o, 1) -> [None, None, None], None], None], None], None]], (r, 0) -> [None, (o, 0) -> [(a, 0) -> [None, (x, 0) -> [(l, 1) -> [None, None, None], (a, 1) -> [None, None, None], None], (i, 0) -> [None, (t, 0) -> [None, (t, 0) -> [None, (o, 1) -> [None, None, None], None], None], None]], (f, 1) -> [None, None, (r, 1) -> [(n, 1) -> [None, None, None], None, None]], (u, 0) -> [None, (n, 2) -> [None, None, None], None]], (z, 0) -> [(s, 0) -> [None, (p, 0) -> [(i, 0) -> [(c, 0) -> [None, (u, 0) -> [(a, 0) -> [None, (r, 0) -> [None, (o, 0) -> [None, (o, 0) -> [None, (r, 1) -> [None, None, None], None], None], None], None], (n, 0) -> [None, (g, 0) -> [None, (a, 1) -> [None, None, None], None], None], None], None], (s, 0) -> [(m, 0) -> [None, (b, 1) -> [None, None, None], None], (s, 1) -> [None, None, None], None], (m, 0) -> [None, (a, 0) -> [None, (p, 1) -> [None, None, None], None], None]], (i, 0) -> [None, (p, 0) -> [None, (r, 0) -> [None, (e, 0) -> [None, (s, 1) -> [None, None, None], None], None], (x, 1) -> [None, None, None]], None], (t, 0) -> [None, (i, 0) -> [None, (o, 0) -> [None, (c, 0) -> [None, (r, 0) -> [None, (a, 1) -> [None, None, None], None], None], None], (o, 0) -> [None, (p, 0) -> [None, (a, 1) -> [None, None, None], None], (r, 0) -> [None, (a, 0) -> [None, (t, 1) -> [None, None, None], None], (u, 0) -> [None, (j, 0) -> [(d, 1) -> [None, None, None], (a, 1) -> [None, None, None], None], None]]]], (u, 0) -> [None, (n, 0) -> [None, (t, 0) -> [None, (s, 1) -> [None, None, None], None], None], None]]], (t, 0) -> [None, (o, 0) -> [(i, 0) -> [None, (u, 0) -> [None, (r, 1) -> [None, None, None], None], None], (p, 0) -> [None, (r, 0) -> [None, (o, 0) -> [None, (t, 1) -> [None, None, None], None], None], None], None], (u, 0) -> [None, (c, 0) -> [None, (k, 1) -> [None, None, None], (t, 0) -> [(s, 0) -> [(d, 0) -> [None, (l, 0) -> [None, (u, 1) -> [None, None, None], None], None], (t, 0) -> [None, (r, 0) -> [None, (a, 0) -> [None, (g, 0) -> [None, (g, 0) -> [None, (a, 1) -> [None, None, None], None], None], None], None], None], None], (a, 1) -> [None, None, (l, 0) -> [None, (u, 1) -> [None, None, None], None]], None]], (w, 0) -> [(v, 0) -> [None, (i, 0) -> [None, (d, 0) -> [None, (z, 0) -> [None, (o, 1) -> [None, None, None], None], None], (u, 0) -> [None, (i, 0) -> [None, (r, 1) -> [None, None, None], None], None]], None], (i, 0) -> [(e, 0) -> [None, (s, 0) -> [None, (t, 0) -> [None, (a, 0) -> [None, (n, 0) -> [None, (t, 1) -> [None, None, None], None], None], None], None], None], (s, 0) -> [None, (o, 0) -> [None, (s, 1) -> [None, None, None], None], None], (o, 0) -> [None, (i, 0) -> [None, (l, 0) -> [None, (d, 0) -> [None, (r, 0) -> [None, (u, 1) -> [None, None, None], None], None], None], None], None]], None]]]], (e, 0) -> [None, (s, 0) -> [(n, 0) -> [None, (n, 0) -> [None, (a, 0) -> [None, (t, 0) -> [None, (y, 1) -> [None, None, None], None], None], None], None], (h, 1) -> [None, None, None], None], None], None]]]]",
        wt100.size == 100
    )

def test_add_word():
    (a15,b15) = words15_add()
    b = test_message2("add",15,[a15,b15],["mismatch when printing tree","mismatch in tree size"])
    (a15e,b15e) = words15_addEmpty()
    c = test_message2("add: empty string (\"\")",15,[a15e,b15e],["mismatch when printing tree","mismatch in tree size"])
    (a100,b100) = words100_add()
    a = test_message2("add",100,[a100,b100],["mismatch when printing tree","mismatch in tree size"])
    return a and b and c

####################################################################################################
# TEST count.1)Count the occurrences for 10 words in.2) Count the occurrences for 10 words not in. #
####################################################################################################
words_in_15 = ["room", "carbos", "run", "meter", "cart", "cage", "phone", "stat", "visas", "up"]
words_in_100  = ["bick", "spipres", "cor", "coll", "run", "fess", "pus", "ritto", "gomd", "ain"]
words_notin = ["carbs", "cat", "cartridge", "zoology", "pop", "alphabet", "ball", "danger", "turnstile", "tree"]

def test15_count_word():
    return [wt15.count(i) for i in words_in_15] == [1, 1, 1, 1, 2, 1, 1, 1, 1, 1]
def test15_notcount_word():
    return [wt15.count(i) for i in words_notin] == [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
def test100_count_word():
    return [wt100.count(i) for i in words_in_100] == [1, 1, 2, 1, 2, 1, 2, 1, 1, 1]
def test100_notcount_word():
    return [wt100.count(i) for i in words_notin] == [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

def test_count_word():
    b = test_message2("count",15,[test15_count_word(),test15_notcount_word()],["mismatch in numbers in tree","mismatch in numbers not in tree"])
    a = test_message2("count",100,[test100_count_word(),test100_notcount_word()],["mismatch in numbers in tree","mismatch in numbers not in tree"])
    return a and b

def test_remove_word():
    for word in words15:
        wt15.remove(word)
        
    for word in words100:
        wt100.remove(word)
        
    print("TESTING Remove ***********************")    
    print("wt15: " + str(wt15))
    print("wt100: " + str(wt100))
    print(wt15.root == None and wt100.root == None)

###################################################
# TESTING all functions. TESTS add, count for now #
###################################################
def test_WordTree(): #TRUE IF PASSES ALL TESTS
    t = time.time()
    a = test_add_word()
    b = test_count_word()
    test_remove_word()
    print("Time in ms: " + str(1000*(time.time()-t)))  
    return a and b

test_WordTree()

[adding 15 words]...........[success]
Min word in wt15 is: cage
***********************************
TESTING WordTree [add] FOR 15 WORDS
[printing results].........[success: passed all tests]

******************************************************
TESTING WordTree [add: empty string ("")] FOR 15 WORDS
[printing results].........[success: passed all tests]

[adding 100 words]...........[success]
Min word in wt100 is: abrar
************************************
TESTING WordTree [add] FOR 100 WORDS
[printing results].........[success: passed all tests]

*************************************
TESTING WordTree [count] FOR 15 WORDS
[printing results].........[success: passed all tests]

**************************************
TESTING WordTree [count] FOR 100 WORDS
[printing results].........[success: passed all tests]

TESTING Remove ***********************
wt15: None
wt100: None
True
Time in ms: 37.89496421813965


True